In [1]:
import Robogame as rg
import networkx as nx
import time, json
import altair as alt
import pandas as pd
import numpy as np

# Simulator
This is an example of a simulator. It will create bets for the two players and the just see what happens.

I'm assuming two teams... one with the secret "alice" and one with "bob." 

To launch the server for this you can do the following in your server directory:

```python api.py -d ./example1 -t1s bob -t2s alice --matchsave bobvalice.json examplematch1```

This will launch the example1match from the example1 directory, using the two secrets (alice and bob). The log of the game will get saved to bobvalice.json which might be useful for analysis after the fact.

In [2]:
# let's read the real match data... this will give us the *real* answers
# you can modify this for different example files but remember to also launch the right server version
robotdata = pd.read_csv("../server/example1/examplematch1.robotdata.csv")

In [3]:
# let's create objects for our two teams
team1 = rg.Robogame("bob")
team2 = rg.Robogame("alice")

In [4]:
# tell the server we're ready
print(team1.setReady())
print(team2.setReady())

{'Result': 'OK'}
{'Result': 'OK'}


In [5]:
# wait for the match to launch
while(True):
    gametime = team1.getGameTime()
    timetogo = gametime['gamestarttime_secs'] - gametime['servertime_secs']
    if ('Error' in gametime):
        print("Error"+str(gametime))
        break
    if (timetogo <= 0):
        print("Let's go!")
        break
    
    print("waiting to launch... game will start in " + str(int(timetogo)))
    time.sleep(1) # sleep 1 second at a time, wait for the game to start

waiting to launch... game will start in 9
waiting to launch... game will start in 8
waiting to launch... game will start in 7
waiting to launch... game will start in 6
waiting to launch... game will start in 5
waiting to launch... game will start in 4
waiting to launch... game will start in 3
waiting to launch... game will start in 2
waiting to launch... game will start in 1
waiting to launch... game will start in 0
Let's go!


In [6]:
# both teams initially bet 50
omniplayerAllBets1 = {}
omniplayerAllBets2 = {}

for i in np.arange(0,100):
    omniplayerAllBets1[int(i)] = int(50)
    omniplayerAllBets2[int(i)] = int(50)
    
team1.setBets(omniplayerAllBets1)
team2.setBets(omniplayerAllBets2)

{'Result': 'OK'}

In [7]:
# let's try to figure out some strategy for each robot based on the real data
# this runs a dumb simulation. It basically knows the truth but it will guess a random number
# around the truth that is proportional to the number of hints it knows about each robot
# the more hints it gets about a robot, the closer it will guess to the truth
#
# the two teams have slightly different amounts of random distributions so in theory
# team 2 will converge on the true answer faster than team 1

lasthints1 = {}
lasthints2 = {}

for timeloop in np.arange(0,100):
    # load some new hints
    team1.getHints()
    team2.getHints()
    
    omniplayerAllBets1 = {}
    omniplayerAllBets2 = {}

    x = team1.getRobotInfo()
    t = team1.getGameTime()
    if 'curtime' in t:
        if (t['curtime'] >= 100):
            break
    else:
        break
      
    # grab the robots that we can guess for
    
    
    # lets check all the hints we've seen so far
    t1hints = team1.getAllPredictionHints()
    t2hints = team2.getAllPredictionHints()

    hints1 = {}
    hints2 = {}
    
    # how many hints do we have per robot?
    for t1 in t1hints:
        id1 = int(t1['id'])
        if (id1 not in hints1):
            hints1[id1] = 1
        else:
            hints1[id1] = hints1[id1] + 1
        
    
    
    for t1 in t2hints:
        id1 = int(t1['id'])
        if (id1 not in hints2):
            hints2[id1] = 1
        else:
            hints2[id1] = hints2[id1] + 1
    
    #print(hints1,hints2)
    
    for row in robotdata[robotdata.id < 100].sort_values('id').iterrows():

        row = row[1]                # get the row data
        expires = int(row.expires)  # figure out when the robot expires
        col = "t_"+str(expires)     # look up the value at that time
        valAtExp = row[col]         # this is the true value
        
        if (expires <= t['curtime']):
            continue                   # skip expired robots

        rowid = int(row.id)

        if (rowid in hints1):
            if ((rowid not in lasthints1) or (hints1[rowid] != lasthints1[rowid])):
                # we have some evidence for this robot
                # pick a random number around the true answer (normal distrib with mean of valAtExp, stdev of 20)
                val = int(np.random.normal(valAtExp,20/hints1[rowid]))
                if (val <  0):
                    val = 0
                if (val > 100):
                    val = 100
                #print(row.id)
                omniplayerAllBets1[rowid] = val

        if (rowid in hints2):
             if ((rowid not in lasthints2) or (hints2[rowid] != lasthints2[rowid])):
                # pick a random number around the true answer (normal distrib with mean of valAtExp, stdev of 10)
                # this will be for team2 which are going to be slightly more accurate
                val = int(np.random.normal(valAtExp,10/hints2[rowid]))
                if (val <  0):
                    val = 0
                if (val > 100):
                    val = 100

                omniplayerAllBets2[rowid] = val

    lasthints1 = hints1
    lasthints2 = hints2
    print("team 1 bets",omniplayerAllBets1)
    print("team 2 bets",omniplayerAllBets2)
    team1.setBets(omniplayerAllBets1)
    team2.setBets(omniplayerAllBets2)
    print("time: ",t['curtime'])
    print(x[['winner','Productivity']].groupby(['winner']).sum())

    time.sleep(6)

team 1 bets {}
team 2 bets {}
time:  0.05
        Productivity
winner              
-2               0.0
team 1 bets {77: 100, 78: 44, 83: 25, 92: 92, 93: 96, 96: 50}
team 2 bets {16: 98, 22: 97, 38: 91, 39: 43, 60: 6, 68: 0, 69: 51, 70: 78, 98: 95}
time:  1.07
        Productivity
winner              
-2               0.0
team 1 bets {15: 10, 16: 100, 21: 55, 23: 40, 44: 86, 46: 85, 77: 100, 78: 26, 83: 41, 92: 89, 93: 85, 96: 62}
team 2 bets {15: 34, 16: 96, 22: 89, 38: 83, 39: 48, 44: 86, 46: 79, 59: 0, 60: 3, 68: 5, 69: 67, 70: 98, 79: 90, 84: 51, 88: 41, 92: 100, 98: 100}
time:  2.09
        Productivity
winner              
-2               0.0
team 1 bets {10: 41, 13: 100, 14: 67, 15: 26, 16: 100, 21: 54, 23: 42, 44: 98, 46: 91, 48: 89, 53: 43, 78: 28, 93: 100, 96: 60}
team 2 bets {2: 87, 3: 76, 7: 50, 15: 28, 16: 92, 28: 0, 44: 94, 46: 90, 53: 79, 59: 2, 67: 72, 70: 97, 79: 66, 83: 28, 84: 44, 88: 38, 92: 100}
time:  3.11
        Productivity
winner              
-2            

team 1 bets {5: 51, 6: 89, 22: 98, 44: 90, 50: 63, 54: 19, 71: 58, 72: 83, 73: 87, 74: 76, 83: 25, 85: 0}
team 2 bets {6: 88, 22: 100, 23: 35, 33: 67, 35: 95, 43: 47, 50: 61, 61: 33, 67: 70, 77: 100, 95: 56}
time:  27.65
        Productivity
winner              
-2          0.000000
 1        118.388418
 2        238.076647
team 1 bets {6: 89, 17: 75, 43: 54, 44: 93, 50: 64, 58: 49, 71: 65, 73: 97, 78: 27, 83: 25, 86: 35}
team 2 bets {6: 91, 22: 99, 31: 56, 50: 63, 52: 41, 61: 28, 67: 75, 75: 100, 76: 54, 77: 100, 95: 56}
time:  28.68
        Productivity
winner              
-2          0.000000
 1        118.388418
 2        487.044153
team 1 bets {6: 89, 9: 51, 10: 30, 17: 76, 23: 33, 25: 72, 27: 93, 43: 51, 58: 44, 69: 62, 72: 68, 78: 23, 86: 29, 90: 91, 94: 98}
team 2 bets {7: 44, 31: 54, 52: 44, 61: 28, 75: 98, 76: 53, 94: 93, 95: 56}
time:  29.71
        Productivity
winner              
-2          0.000000
 1        118.388418
 2        605.994024
team 1 bets {6: 88, 9: 51, 10

team 1 bets {23: 33, 50: 62, 67: 72, 76: 53, 78: 19, 89: 63}
team 2 bets {11: 26, 14: 66, 38: 85, 74: 74, 89: 60, 96: 68}
time:  57.56
        Productivity
winner              
-2          0.000000
 1        627.283843
 2       1934.077105
team 1 bets {7: 44, 23: 36, 33: 66, 50: 64, 67: 75, 81: 58, 88: 34, 96: 67}
team 2 bets {14: 64, 38: 87, 55: 23, 60: 2, 85: 0, 89: 59, 96: 67}
time:  58.59
        Productivity
winner              
-2          0.000000
 1        627.283843
 2       1986.749292
team 1 bets {7: 42, 50: 64, 65: 56, 78: 19, 81: 61, 82: 4, 88: 35, 96: 66}
team 2 bets {7: 42, 55: 25, 60: 3, 83: 26, 85: 0}
time:  59.63
        Productivity
winner              
-2          0.000000
 1        627.283843
 2       2021.016975
team 1 bets {11: 23, 65: 58, 67: 75, 78: 20, 82: 4, 84: 44}
team 2 bets {7: 43, 32: 62, 83: 23, 91: 9}
time:  60.66
        Productivity
winner              
-2          0.000000
 1        627.283843
 2       2094.250801
team 1 bets {11: 27, 60: 5, 67: 75,

In [8]:
# that's more or less it... you can modify the guesses us different strategies
# for example, you can also consider the network